In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from fbprophet import Prophet
from datetime import datetime
from dateutil import parser
from statsmodels.tsa.stattools import adfuller


In [2]:
inv = pd.read_csv(r"C:\Users\Premnath\OneDrive\Study\DDS\DDS_Barkawi_project\MMM_Data\skus.csv", sep =';')
order = pd.read_csv(r"C:\Users\Premnath\OneDrive\Study\DDS\DDS_Barkawi_project\MMM_Data\customer_orders.csv", sep =';')

In [3]:
order.head()

,Order Number,Order Line,Outlet,Part,Creation Date,Requested Date,Shipped Date,Requested Quantity,Shipped Quantity
0,Order_01,1,1,8WW91511052540M,14.01.2017 00:00,14.01.2017,14.01.2017,-1.0,-1.0
1,Order_02,3,1,6YX87639578537D,07.01.2017 00:00,07.01.2017,09.01.2017,12.0,12.0
2,Order_02,7,1,4RQ12578481053J,30.12.2016 00:00,30.12.2016,30.12.2016,2.0,2.0
3,Order_03,1,1,2DS35615142379C,29.12.2016 00:00,29.12.2016,29.12.2016,3.0,3.0
4,Order_03,2,1,5DG98783853149R,27.12.2016 00:00,27.12.2016,06.01.2017,8.0,8.0


In [6]:
d = order[order['Part']=='9ZW54628430295V']['Requested Date'].max()
d

'19.05.2019'

In [7]:
print("Size of order data is ", order.shape , "Size of inventory data is ", inv.shape)

Size of order data is  (33696, 9) Size of inventory data is  (37760, 5)


In [7]:
zero = order[order['Requested Quantity']==0].index
neg = order[order['Requested Quantity']<0].index
print(zero, neg)
order.drop(zero, axis=0, inplace=True)
order.drop(neg, axis=0, inplace=True)

Int64Index([  142,   154,   285,   406,   478,   583,   884,   895,   956,
              985,
            ...
            33139, 33184, 33326, 33333, 33336, 33356, 33437, 33438, 33595,
            33638],
           dtype='int64', length=858) Int64Index([    0,    16,   359,   738,   764,   886,  1145,  1188,  1273,
             1386,
            ...
            33123, 33185, 33258, 33525, 33579, 33580, 33581, 33582, 33583,
            33584],
           dtype='int64', length=385)


In [8]:
order.rename(columns={'Requested Date': 'Requested_Date', 'Creation Date': 'Creation_Date',
                      'Shipped Date':'Shipped_Date','Order Number': 'Order_Number', 'Order Line': 'Order_Line',
                      'Requested Quantity':'Requested_Quantity', 'Shipped Quantity':'Shipped_Quantity'}, inplace=True)


In [9]:
order.head()

,Order_Number,Order_Line,Outlet,Part,Creation_Date,Requested_Date,Shipped_Date,Requested_Quantity,Shipped_Quantity
0,Order_01,1,1,8WW91511052540M,14.01.2017 00:00,14.01.2017,14.01.2017,-1.0,-1.0
1,Order_02,3,1,6YX87639578537D,07.01.2017 00:00,07.01.2017,09.01.2017,12.0,12.0
2,Order_02,7,1,4RQ12578481053J,30.12.2016 00:00,30.12.2016,30.12.2016,2.0,2.0
3,Order_03,1,1,2DS35615142379C,29.12.2016 00:00,29.12.2016,29.12.2016,3.0,3.0
4,Order_03,2,1,5DG98783853149R,27.12.2016 00:00,27.12.2016,06.01.2017,8.0,8.0


In [10]:
print("Size of order data is ", order.shape , "Size of inventory data is ", inv.shape)

Size of order data is  (33696, 9) Size of inventory data is  (37760, 5)


In [11]:
order.describe()

,Order_Line,Outlet,Requested_Quantity,Shipped_Quantity
count,33696.000000,33696.000000,33696.000000,33696.000000
mean,14.813598,4.675392,3.255758,3.255194
std,151.108684,3.082693,6.479469,6.477098
min,1.000000,1.000000,-125.000000,-125.000000
25%,1.000000,2.000000,1.000000,1.000000
50%,3.000000,4.000000,2.000000,2.000000
75%,8.000000,7.000000,2.000000,2.000000
max,5003.000000,10.000000,400.000000,400.000000


In [12]:
order.describe(include = ['O']) # Capital O

,Order_Number,Part,Creation_Date,Requested_Date,Shipped_Date
count,33696,33696,33696,33696,33696
unique,23080,3776,1111,1107,1151
top,Order_10286,7UI86399152671N,19.05.2018 00:00,30.04.2018,11.07.2019
freq,47,1172,60,56,60


In [13]:
order['Requested_Date']= pd.to_datetime(order['Requested_Date'],format='%d.%m.%Y')
order['Shipped_Date']= pd.to_datetime(order['Shipped_Date'],format='%d.%m.%Y')
order['Creation_Date']= pd.to_datetime(order['Creation_Date'],format='%d.%m.%Y %H:%M')

In [14]:
order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33696 entries, 0 to 33695
Data columns (total 9 columns):
Order_Number          33696 non-null object
Order_Line            33696 non-null int64
Outlet                33696 non-null int64
Part                  33696 non-null object
Creation_Date         33696 non-null datetime64[ns]
Requested_Date        33696 non-null datetime64[ns]
Shipped_Date          33696 non-null datetime64[ns]
Requested_Quantity    33696 non-null float64
Shipped_Quantity      33696 non-null float64
dtypes: datetime64[ns](3), float64(2), int64(2), object(2)
memory usage: 2.3+ MB


In [15]:
order['Delay']=(order['Shipped_Date'] - order['Requested_Date']).dt.days

In [16]:
order['Requested_Month'] = pd.DatetimeIndex(order['Requested_Date']).month

In [17]:
order['Requested_Year'] = pd.DatetimeIndex(order['Requested_Date']).year

In [18]:
order.head()

,Order_Number,Order_Line,Outlet,Part,Creation_Date,Requested_Date,Shipped_Date,Requested_Quantity,Shipped_Quantity,Delay,Requested_Month,Requested_Year
0,Order_01,1,1,8WW91511052540M,2017-01-14,2017-01-14,2017-01-14,-1.0,-1.0,0,1,2017
1,Order_02,3,1,6YX87639578537D,2017-01-07,2017-01-07,2017-01-09,12.0,12.0,2,1,2017
2,Order_02,7,1,4RQ12578481053J,2016-12-30,2016-12-30,2016-12-30,2.0,2.0,0,12,2016
3,Order_03,1,1,2DS35615142379C,2016-12-29,2016-12-29,2016-12-29,3.0,3.0,0,12,2016
4,Order_03,2,1,5DG98783853149R,2016-12-27,2016-12-27,2017-01-06,8.0,8.0,10,12,2016


In [19]:
order.groupby('Requested_Year')['Order_Number'].value_counts()

Requested_Year  Order_Number
2016            Order_14        3
                Order_03        2
                Order_04        2
                Order_02        1
                Order_05        1
                               ..
2020            Order_4835      1
                Order_6686      1
                Order_6687      1
                Order_7582      1
                Order_7583      1
Name: Order_Number, Length: 23217, dtype: int64

In [20]:
# Total number of orders processed per year
table = pd.pivot_table(order, values='Order_Number', index=['Requested_Year'],
                     aggfunc='count')

In [21]:
table

,Order_Number
Requested_Year,
2016,47
2017,10924
2018,11192
2019,11504
2020,29


In [22]:
inv.head()

,Part,Outlet,Inventory Qty,Costs,MOQ
0,2MZ25020436304R,1,0,3.16,10
1,10MK66232719455Q,1,0,6.24,10
2,8XW94040648199M,1,0,4.13,10
3,10KX70570551177B,1,2,4.03,10
4,7ET14701568892K,1,2,4.82,10


In [23]:
inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37760 entries, 0 to 37759
Data columns (total 5 columns):
Part             37760 non-null object
Outlet           37760 non-null int64
Inventory Qty    37760 non-null int64
Costs            37760 non-null float64
MOQ              37760 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 1.4+ MB


In [24]:
inv.describe(include = ['O']) # Capital O

,Part
count,37760
unique,3776
top,9ZA97853471928O
freq,10


In [26]:
unique_parts = list(order['Part'].unique())
unique_parts#[0:3]

['8WW91511052540M',
 '6YX87639578537D',
 '4RQ12578481053J',
 '2DS35615142379C',
 '5DG98783853149R',
 '6TL87424290587L',
 '2CG15479205640M',
 '5MI17503293098G',
 '10LT38882262598Q',
 '9DG12017945528Y',
 '9BB71218385038A',
 '10OD57818576174M',
 '9EN21705838310Q',
 '10HV68699525208Z',
 '10SO55529479938E',
 '1UY99988115496W',
 '3OB35915287851N',
 '4DP79230565166R',
 '8XB76584829115D',
 '10TC52557647378J',
 '9QA73561375619Q',
 '2YJ13060377870Z',
 '1ZV68217703910Y',
 '7UI86399152671N',
 '7NJ80148957966H',
 '4EX30371760759I',
 '6FY82551363568B',
 '5RQ37652507562Q',
 '2GS55289964768T',
 '4ZT23274659474R',
 '10CJ75739259300L',
 '3LX36135773883K',
 '3PV59838098006F',
 '8WE83858329899J',
 '2OG86503939991X',
 '3LH38902604243H',
 '3SD84165389689K',
 '1RU69363915794G',
 '9MF38310372355F',
 '1QJ74661681415Y',
 '5EC16756557643G',
 '2HW22784704592E',
 '5MZ36579729040E',
 '1ZE48092510400K',
 '1LI42225007355C',
 '1ND11428801930Q',
 '7TW92788796837I',
 '10DG77058073922H',
 '7ZV82796398332K',
 '3PN47687881

In [26]:
#order['Part'] = order['Part'].str.replace(' ','')

In [40]:
#d = order[order['Part']=='9ZW54628430295V']['Requested_Date'].max()
#d

In [39]:
#o = list(order.loc[(order['Part']=='10MK66232719455Q') & (order['Requested_Date']== d)]['Outlet'])[0]
#o

In [34]:
#costs = inv.loc[(inv['Outlet']==o) & (inv['Part']=='10MK66232719455Q')]['Costs']
#costs = list(costs)[0]
#Price = ['10MK66232719455Q',
#costs

In [36]:
#costs

In [28]:
d = order[order['Part']=='9ZW54628430295V']['Requested_Date'].max()
d

Timestamp('2019-05-19 00:00:00')

In [29]:
selling_cost = []
for i in unique_parts:
    Price =[]
    #print(i)
    d = order[order['Part']==i]['Requested_Date'].max()
    #print(d)
    # we are extracting only the value we want from the data frame by converting to list
    o = list(order.loc[(order['Part']==i) & (order['Requested_Date']== d)]['Outlet'])[0]
    #print(o)
    # we are extracting only the value we want from the data frame by converting to list
    cost = list(inv.loc[(inv['Outlet']==o) & (inv['Part']==i)]['Costs'])[0]
    #print(cost)
    Price = [i, cost]
    selling_cost.append(Price)



In [31]:
selling_cost

[['8WW91511052540M', 40.53],
 ['6YX87639578537D', 0.37],
 ['4RQ12578481053J', 35.4],
 ['2DS35615142379C', 1.2],
 ['5DG98783853149R', 2.36],
 ['6TL87424290587L', 3.9],
 ['2CG15479205640M', 0.24],
 ['5MI17503293098G', 1.64],
 ['10LT38882262598Q', 2.9],
 ['9DG12017945528Y', 7.67],
 ['9BB71218385038A', 1.81],
 ['10OD57818576174M', 0.78],
 ['9EN21705838310Q', 15.09],
 ['10HV68699525208Z', 447.91],
 ['10SO55529479938E', 1.14],
 ['1UY99988115496W', 11.13],
 ['3OB35915287851N', 8.32],
 ['4DP79230565166R', 32.64],
 ['8XB76584829115D', 42.58],
 ['10TC52557647378J', 16.72],
 ['9QA73561375619Q', 3.16],
 ['2YJ13060377870Z', 6.09],
 ['1ZV68217703910Y', 4.97],
 ['7UI86399152671N', 24.03],
 ['7NJ80148957966H', 17.14],
 ['4EX30371760759I', 11.65],
 ['6FY82551363568B', 10.21],
 ['5RQ37652507562Q', 0.45],
 ['2GS55289964768T', 3.21],
 ['4ZT23274659474R', 46.98],
 ['10CJ75739259300L', 0.52],
 ['3LX36135773883K', 1.11],
 ['3PV59838098006F', 3.21],
 ['8WE83858329899J', 12.27],
 ['2OG86503939991X', 0.81],
 ['

In [32]:
price_df = pd.DataFrame(selling_cost)

In [33]:
price_df.to_csv('prices.csv', index=False)

In [42]:
# Extract the data of particular outlet
def outlet_num(n):
    return order[order['Outlet']==n]

In [43]:
# Total yearly sales of particular outlet
def sum_sales(a, year):
    v = a[a['Requested_Year']== year]['Order_Number'].count()
    return v

In [44]:
# # Total yearly sales of all individual outlets
sales = []
for i in range(1,11):
    total_sales = []
    for year in [2017,2018,2019]:        
        tot_sales = sum_sales(outlet_num(i), year)
        total_sales.append(tot_sales)
    sales.append(total_sales)
    

In [45]:
sales

[[2418, 2481, 2420],
 [1219, 1266, 1299],
 [1385, 1380, 1317],
 [719, 697, 712],
 [403, 509, 634],
 [430, 369, 543],
 [1369, 1480, 1263],
 [1042, 1119, 1274],
 [889, 778, 788],
 [644, 735, 796]]

In [46]:
Sales_Details = pd.DataFrame(sales, columns=['2017','2018','2019'])
Sales_Details['Outlets'] = ['ou_1','ou_2','ou_3','ou_4','ou_5','ou_6','ou_7','ou_8','ou_9','ou_10']
Sales_Details = Sales_Details[['Outlets','2017','2018','2019']]

In [47]:
Sales_Details

,Outlets,2017,2018,2019
0,ou_1,2418,2481,2420
1,ou_2,1219,1266,1299
2,ou_3,1385,1380,1317
3,ou_4,719,697,712
4,ou_5,403,509,634
5,ou_6,430,369,543
6,ou_7,1369,1480,1263
7,ou_8,1042,1119,1274
8,ou_9,889,778,788
9,ou_10,644,735,796


In [ ]:
sales_Details.iloc[:,:].sum()

In [ ]:
sns.lineplot(data = sales_Details, x='Outlets', y='2019')

In [ ]:
# sales of all the outlets in a year
def yearly_sales(year, sales):
    ind = year-2017
    l = []
    for i in range(10):
        v = sales[i][ind]
        l.append(v)
    return l

In [ ]:
yearly_sales(2017,sales)

In [ ]:
# Total sales in a year

def total_sales(sales):
    l =[]
    for i in range(3):
        add = 0
        for j in range(10):
            v = sales[j][i]
            add += v
        l.append(add)
    return l

In [ ]:
total_sales(sales)

In [ ]:
sales

In [ ]:
def outlet_total(Sales_Details):
    KL = []
    for k in range(0,10):
        s =  Sales_Details.iloc[k,1:].sum()
        KL.append(s)
    return KL

In [ ]:
outlet_sales = pd.DataFrame(outlet_total(Sales_Details))

In [ ]:
outlet_sales['Outlets'] = ['outlet_1','outlet_2','outlet_3','outlet_4','outlet_5','outlet_6','outlet_7','outlet_8','outlet_9','outlet_10']

In [ ]:
outlet_sales.rename(columns={0:'Orders'})

In [ ]:
df = order[order['Part']== '7UI86399152671N']

In [ ]:
df = df[df['Outlet']== 1]

In [ ]:
df.head()

In [ ]:
df16 = df[df['Requested_Year'] == 2017]

In [ ]:
def cum_mon(year):
    df_year = df[df['Requested_Year']==year]
    cum_year = []
    month_no = []
    for i in range(1,13):
        q = df_year[df_year['Requested_Month']== i]['Requested_Quantity'].sum()
        cum_year.append(q)
        month_no.append(i)
    return cum_year,month_no
    #sns.lineplot(x=list(range(1,13)), y=cum_2017)

In [ ]:
def cum():
    S = []
    M = []
    for year in [2017,2018,2019]:
        print(year)
        
        sales,mo = cum_mon(year)
        print(sales)
        for j in range(12):
            S.append(sales[j])
            M.append(mo[j])
    sns.lineplot(x=list(range(1,37)),y=S)

In [ ]:
cum()

In [ ]:
# Train test split

train_size = int(len(order) * 0.66)
order_train, order_test = order[0:train_size], order[train_size:len(order)]
print('Observations: %d' % (len(order)))
print('Training Observations: %d' % (len(order_train)))
print('Testing Observations: %d' % (len(order_test)))

In [ ]:
order_train.head()

In [ ]:
or_tr_da = order_train[(order_train['Part'] == '7UI86399152671N') & (order_train['Outlet']==1)]

In [ ]:
or_tr_da = or_tr_da[['Requested_Date','Requested_Quantity']]
or_tr_da.info()

In [ ]:
or_tr_da['ds'] = or_tr_da['Requested_Quantity']
or_tr_da['y'] = or_tr_da['Requested_Date']

In [ ]:
or_tr_da.drop(columns = ['Requested_Date','Requested_Quantity'],axis=1, inplace = True)

In [ ]:
or_tr_da['year']=or_tr_da['y'].apply(lambda x: str(x).split('-')[0])
or_tr_da['month']=or_tr_da['y'].apply(lambda x: str(x).split('-')[1])
or_tr_da['time'] = or_tr_da['year'] + '-' + or_tr_da['month']
or_tr_da.drop(['y','year','month'], axis=1, inplace=True)
or_tr_da.rename(columns={'time':'y'}, inplace=True)

#trai = pd.DataFrame(or_tr_da.groupby(by=['year','month']).agg(['ds', 'y', 'year', 'month']))

In [ ]:
or_tr_da['y']= pd.to_datetime(or_tr_da['y'],format='%Y-%m')

In [ ]:
or_tr_da.head(20)

In [ ]:
or_tr_da.groupby(pd.Grouper(key='y',freq='M'))['ds'].apply(sum)

In [ ]:
or_tr_da.groupby('y')['ds'].transform(lambda x: sum(x))

In [ ]:
or_tr_da.groupby(['y']).agg(or_tr_da['ds', 'y' ])

In [ ]:
m = Prophet(yearly_seasonality = True,weekly_seasonality= True, interval_width=0.95)
m.fit(or_tr_da)
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)